In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import pywt
import math
import time
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from pytorchtools import EarlyStopping

import scipy.io as sio 
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,cohen_kappa_score

In [2]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'/media/dy113/disk1/Project_xjr/dataset')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'IndianPines/Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'IndianPines/Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas/Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas/Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU/PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU/PaviaU_gt.mat'))['paviaU_gt']
    elif name == 'KSC':
        data = sio.loadmat(os.path.join(data_path, 'KSC/KSC.mat'))['KSC']
        labels = sio.loadmat(os.path.join(data_path, 'KSC/KSC_gt.mat'))['KSC_gt']
    elif name == 'DISPU':
        data = sio.loadmat(os.path.join(data_path, 'DISPU/PUtrain_gt.mat'))['PU_tgt']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU/PaviaU_gt.mat'))['paviaU_gt']
    return data, labels

In [3]:
def padWithZeros(X,margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2 * margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    # 给 X 做 padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test

In [4]:
class_num = 16
dataset = 'SA'
batchsize = 64
X, y = loadData(dataset)
print(X.shape)
print(y.shape)

test_ratio = 0.99
patch_size = 15
band_nums = X.shape[2]
pca_components = 15

print('Hyperspectral data shape: ', X.shape)
print('Label shape: ',y.shape)
print('Band nums: ', X.shape[2])

(512, 217, 204)
(512, 217)
Hyperspectral data shape:  (512, 217, 204)
Label shape:  (512, 217)
Band nums:  204


In [5]:
def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX,(X.shape[0],X.shape[1],numComponents))
    return newX

In [6]:
X_pca = applyPCA(X,numComponents=pca_components)
print('Data shape after PCA: ',X_pca.shape)

X_pca,y = createImageCubes(X_pca,y,windowSize=patch_size)
print('Data cube X shape: ', X_pca.shape)
print('Data cube y shape: ', y.shape)


# X,y = createImageCubes(X,y,windowSize=patch_size)
# print('Data cube X shape: ', X.shape)
# print('Data cube y shape: ', y.shape)

dim3=X_pca.shape[3]
print(dim3)
band_nums = dim3


Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
# Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)
print('Xtrain shape: ',Xtrain.shape)
print('Xtest shape: ',Xtest.shape)
print('ytrain shape: ',ytrain.shape)
print('ytest shape: ',ytest.shape)

Xtrain = Xtrain.reshape(-1, patch_size, patch_size, Xtrain.shape[3], 1)
Xtest  = Xtest.reshape(-1, patch_size, patch_size, Xtrain.shape[3], 1)
print('before transpose: Xtrain shape: ', Xtrain.shape)
print('before transpose: Xtest  shape: ', Xtest.shape)
print('before transpose: Xtest  shape: ', Xtest.shape)

# Xtrain = Xtrain.transpose(0, 4, 3, 2, 1)
# Xtest  = Xtest.transpose(0, 4, 3, 2, 1)
Xtrain = Xtrain.transpose(0, 4, 2, 1, 3)
Xtest  = Xtest.transpose(0, 4, 2, 1, 3)
print('after transpose: Xtrain shape: ', Xtrain.shape)
print('after transpose: Xtest  shape: ', Xtest.shape)


# Xtrain = Xtrain.transpose(0, 3, 1, 2)
# Xtest  = Xtest.transpose(0, 3, 1, 2)
# print('after transpose: Xtrain shape: ', Xtrain.shape)
# print('after transpose: Xtest  shape: ', Xtest.shape)



class TrainDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtrain.shape[0]
        self.x_data = torch.FloatTensor(Xtrain)
        self.y_data = torch.LongTensor(ytrain)
    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index] 
    def __len__(self):
        # 返回文件数据的数目
        return self.len


class TestDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtest.shape[0]
        self.x_data = torch.FloatTensor(Xtest)
        self.y_data = torch.LongTensor(ytest)
    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        # 返回文件数据的数目
        return self.len
      

trainset = TrainDS()
testset  = TestDS()
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batchsize, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(dataset=testset,  batch_size=batchsize, shuffle=False, num_workers=0)

Data shape after PCA:  (512, 217, 15)
Data cube X shape:  (54129, 15, 15, 15)
Data cube y shape:  (54129,)
15
Xtrain shape:  (541, 15, 15, 15)
Xtest shape:  (53588, 15, 15, 15)
ytrain shape:  (541,)
ytest shape:  (53588,)
before transpose: Xtrain shape:  (541, 15, 15, 15, 1)
before transpose: Xtest  shape:  (53588, 15, 15, 15, 1)
before transpose: Xtest  shape:  (53588, 15, 15, 15, 1)
after transpose: Xtrain shape:  (541, 1, 15, 15, 15)
after transpose: Xtest  shape:  (53588, 1, 15, 15, 15)


In [7]:
class Wavelet(nn.Module):
    """This module extract wavelet coefficient defined in pywt
    and create 3D convolution kernels to be able to use GPU"""

    def _coef_h(self, in_planes, coef):
        """Construct the weight matrix for horizontal 3D convolution.
        The weights are repeated on the diagonal"""
        v = []
        for i in range(in_planes):
            l = []
            for j in range(in_planes):
                if i == j:
                    l.append([[[c for c in coef]]])
                else:
                    l.append([[[0.0 for c in coef]]])
            v.append(l)
#         print(v)
        return v

    def _coef_v(self, in_planes, coef):
        """Construct the weight matrix for vertical 3D convolution.
        The weights are repeated on the diagonal"""
        v = []
        for i in range(in_planes):
            l = []
            for j in range(in_planes):
                if i == j:
                    l.append([[[c] for c in coef]])
                else:
                    l.append([[[0.0] for c in coef]])
            v.append(l)
#         print(v)
        return v

    def _coef_d(self, in_planes, coef):
        """Construct the weight matrix for depth 3D convolution.
        The weights are repeated on the diagonal"""
        v = []
        for i in range(in_planes):
            l = []
            for j in range(in_planes):
                if i == j:
                    l.append([[[c]] for c in coef])
                else:
                    l.append([[[0.0]] for c in coef])
            v.append(l)
#         print(v)
        return v
    
    def __init__(self, in_planes, horizontal, vertical, name="db2"):
        super(Wavelet, self).__init__()

        # Import wavelet coefficients
        import pywt
        wavelet = pywt.Wavelet(name)
        coef_low = wavelet.dec_lo
        coef_high = wavelet.dec_hi
        # Determine the kernel 3D shape
        nb_coeff = len(coef_low)
        # print(nb_coeff)

        if horizontal & (not vertical):
            kernel_size = (1, 1, nb_coeff)
            # stride = (1, 1, 2)
            stride = (1, 1, 1)
            pad = (nb_coeff // 2, nb_coeff - 1 - nb_coeff // 2, 0, 0, 0, 0)
            weights_low = self._coef_h(in_planes, coef_low)
            weights_high = self._coef_h(in_planes, coef_high)

        elif (not horizontal) & vertical:
            kernel_size = (1, nb_coeff, 1)
            # stride = (1, 2, 1)
            stride = (1, 1, 1)
            pad = (0, 0, nb_coeff // 2, nb_coeff - 1 - nb_coeff // 2, 0, 0)
            weights_low = self._coef_v(in_planes, coef_low)
            weights_high = self._coef_v(in_planes, coef_high)

        elif (not horizontal) & (not vertical):
            kernel_size = (nb_coeff, 1, 1)
            # stride = (2, 1, 1)
            stride = (1, 1, 1)
            pad = (0, 0, 0, 0, nb_coeff // 2, nb_coeff - 1 - nb_coeff // 2)
            weights_low = self._coef_d(in_planes, coef_low)
            weights_high = self._coef_d(in_planes, coef_high)
           
        # Create the conv2D
        self.conv_high = nn.Conv3d(
            in_planes, in_planes, kernel_size=kernel_size, stride=stride, bias=False)
        self.conv_low = nn.Conv3d(
            in_planes, in_planes, kernel_size=kernel_size, stride=stride, bias=False)
        self.padding = nn.ReplicationPad3d(padding=pad)

        # Replace their weights
        self.conv_high.weight = torch.nn.Parameter(
            data=torch.Tensor(weights_high), requires_grad=False)
        self.conv_low.weight = torch.nn.Parameter(
            data=torch.Tensor(weights_low), requires_grad=False)

    def forward(self, x):
        '''Returns the approximation and detail part'''
        x = self.padding(x)
        return (self.conv_low(x), self.conv_high(x))

In [8]:
class Wavelet3D(nn.Module):
    def __init__(self, in_planes, name="db2"):
        super(Wavelet3D, self).__init__()
        self.horizontal_wavelet = Wavelet(in_planes, horizontal=True, vertical=False, name=name)
        self.vertical_wavelet = Wavelet(in_planes, horizontal=False, vertical=True,  name=name)
        self.depth_wavelet = Wavelet(in_planes, horizontal=False, vertical=False,  name=name)

    def forward(self, x):
        '''Returns (LL, LH, HL, HH)'''
        (c, d) = self.horizontal_wavelet(x)
        #print('H',c.size(),d.size())
        (LL, LH) = self.vertical_wavelet(c)
        #print('V',LL.size(),LH.size())
        (HL, HH) = self.vertical_wavelet(d)
        (LLL, LLH) = self.depth_wavelet(LL)
        #print('D',LLL.size(),LLH.size())
        (LHL, LHH) = self.depth_wavelet(LH)
        (HLL, HLH) = self.depth_wavelet(HL)
        (HHL, HHH) = self.depth_wavelet(HH)
        return (LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH)    

In [9]:
class waveblock(nn.Module):
    def __init__(self, in_planes, name):
        super(waveblock, self).__init__()
        self.wavelet3d = Wavelet3D(in_planes,name=name)

    def forward(self,x):
        (LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH) = self.wavelet3d(x)
        x = LLL
        details = torch.cat([LLH, LHL, LHH, HLL, HLH, HHL, HHH],1)
        return x,details

In [10]:
class BasicConvBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size,stride,padding=0):
        super(BasicConvBlock, self).__init__()
        self.bn1 = nn.BatchNorm3d(out_planes)
        self.relu = nn.ReLU(inplace=True)    
        self.conv1 = nn.Conv3d(in_planes,out_planes,kernel_size, stride=stride,
                               padding=padding, bias=False)
                   

    def forward(self, x):
        return self.relu(self.bn1(self.conv1(x)))

In [11]:
class BasicConvBlock2D(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size,stride,padding=0):
        super(BasicConvBlock2D, self).__init__()
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes,kernel_size, stride=stride,
                               padding=padding, bias=False)

    def forward(self, x):
        return self.relu(self.bn1(self.conv1(x)))

In [12]:
class WaveCnntest(nn.Module):
    def __init__(self, num_classes,band_nums,patch_size):
        super(WaveCnntest, self).__init__() 
        # self.conv1 = BasicConvBlock2D(band_nums,patch_size,1,1,0)
        
        # self.conv0 = BasicConvBlock(1,1,(1,1,5),(1,1,2),1)
        # self.conv01 = BasicConvBlock(1,1,(1,1,5),(1,1,2),1)
        # self.conv02 = BasicConvBlock(1,1,(1,1,5),(1,1,2),1)

        self.conv00 = BasicConvBlock(1,15,(1,1,7),(1,1,5),(0,0,1))
        self.conv01 = BasicConvBlock(15,15,(1,1,20),(1,1,1),0)
        # self.conv02 = BasicConvBlock(15,15,(1,1,5),(1,1,),1)
        self.conv02 = BasicConvBlock(1,15,(1,1,band_nums),(1,1,1),0)

        self.wave1 = waveblock(1, name="db1")    
#         self.wave2 = waveblock(1, name="db2")
#         self.wave3 = waveblock(1, name="db2")

        self.conv1 = BasicConvBlock(8,8,3,1,1)
        self.conv2 = BasicConvBlock(1,8,3,1,1)

        # self.conv3 = BasicConvBlock(32,32,3,1,1)
                
#         # bottleneck layer        
        self.conv3 = nn.Sequential(
                        BasicConvBlock(8,16,3,1),
                        BasicConvBlock(16,8,3,1))
        
        self.pool = nn.MaxPool3d(kernel_size=3,stride=3)
        
        
    def forward(self,x):
        # x = self.conv00(x)
        # # print(x.shape)
        # # b=x.shape[4]
        # # print(b)
        # x = self.conv01(x)
        # print(x.shape)

        x = self.conv02(x)
        x = x.reshape(x.shape[0],1,x.shape[1],x.shape[1],x.shape[1]) 
        
        # x = self.conv01(x)
        # x = self.conv02(x)
        # x = x.unsqueeze(1)
        # # x = x.permute(0,1,3,4,2)

        app1, det1 = self.wave1(x)
        wave1 = torch.cat([app1, det1],1)
        # x11 = self.conv1(wave1)

         # x1 = self.conv2(x)          
        # x2 = torch.cat([x1, x11],1)
        # x2 = torch.cat([x1, wave1],1)

        x2 = wave1

        x3 = self.conv3(x2)  
        # x3 = self.conv3(x2)        
        # x3 = self.conv4(x2)
        x3 = self.pool(x3)       

        x3 = x3.reshape(x3.shape[0],-1)
                       

#         app2, det2 = self.wave2(app1)
#         wave2 = torch.cat([app2, det2],1)
             
#         x2 = self.conv3(wave2)        
#         x2 = torch.cat([wave2,x2,x1],1)  
        
#         x2 = self.conv4(x2)
#         x2 = x2.reshape(x2.shape[0],-1)
      
#         app2, det2 = self.wave2(app1)
#         wave2 = torch.cat([app2, det2],1)
#         x2 = self.conv2(wave2)
        
#         app3, det3 = self.wave3(app2)
#         wave3 = torch.cat([app3, det3],1)
#         x3 = self.conv2(wave3)
               
#         app4, det4 = self.wave3(app3)
#         wave4 = torch.cat([app4, det4],1)
#         x4 = self.conv2(wave4)
                            
#         x4 = torch.cat([wave1,x1,wave2,x2,wave3,x3,wave4,x4],1)    
# #         print(x3.shape)
#         x4 = self.conv3(x4)
# #         x3 = self.pool(x3)
#         x4 = x4.reshape(x4.shape[0],-1)
# #         flat = x3.shape[1]  
# #         print(flat)
        
        return x3

In [13]:
if __name__ == "__main__":
#     input = torch.randn(1024,band_nums,patch_size, patch_size)
#     input = torch.randn(1, 3, 15, 15, 15)
    # input = torch.randn(1024,1,patch_size, patch_size,pca_components)
    input = torch.randn(1024,1,patch_size, patch_size,dim3)
    m_net = WaveCnntest(class_num,band_nums,patch_size)
    res = m_net(input)
    print('res',res.shape)

res torch.Size([1024, 216])


In [14]:
class WaveCnn(nn.Module):
    def __init__(self, num_classes,band_nums,patch_size):
        super(WaveCnn, self).__init__()  
        # self.conv1 = BasicConvBlock2D(band_nums,patch_size,1,1,0)
        
        self.conv00 = BasicConvBlock(1,15,(1,1,7),(1,1,5),(0,0,1))
        self.conv01 = BasicConvBlock(15,15,(1,1,20),(1,1,1),0)
        self.conv02 = BasicConvBlock(1,patch_size,(1,1,band_nums),(1,1,1),0)

        self.wave1 = waveblock(1, name="db1")    
        self.wave2 = waveblock(1, name="db2")

        self.conv1 = BasicConvBlock(8,8,3,1,1)
        self.conv2 = BasicConvBlock(1,8,3,1,1)
                
#         # bottleneck layer        
        self.conv3 = nn.Sequential(
                        BasicConvBlock(8,16,3,1),
                        BasicConvBlock(16,8,3,1))
        
        self.pool = nn.MaxPool3d(kernel_size=3,stride=3)
        
        self.classifier = nn.Sequential(
                            # nn.Linear(1536,256),
                            nn.Linear(216,128),
                            # nn.Linear(1280,256),
                            nn.Dropout(p=0.4),
                            nn.ReLU(inplace = True),
                            # nn.Linear(256,128),
                            # nn.Dropout(p=0.4),
                            # nn.ReLU(inplace = True),
                            nn.Linear(128,num_classes),
                            nn.Dropout(p=0.4))                     
        
    def forward(self,x):
        # x = self.conv00(x)
        # x = self.conv01(x)
        # # print(x.shape)

        x = self.conv02(x)
        x = x.reshape(x.shape[0],1,x.shape[1],x.shape[1],x.shape[1]) 
           
        # x = self.conv01(x)
        # x = self.conv02(x)
        # x = x.unsqueeze(1)
        # # x = x.permute(0,1,3,4,2)

        app1, det1 = self.wave1(x)
        wave1 = torch.cat([app1, det1],1)
        # x11 = self.conv1(wave1)

        # x1 = self.conv2(x)          
        # x2 = torch.cat([x1, x11],1)
        # x2 = torch.cat([x1, wave1],1)

        x2 = wave1
        
        x3 = self.conv3(x2)        
        x3 = self.pool(x3)       
        x3 = x3.reshape(x3.shape[0],-1)
                       
        out = self.classifier(x3)     

        return out

In [15]:
if __name__ == "__main__":
    # input = torch.randn(1024,1,patch_size, patch_size,pca_components)
    input = torch.randn(1024,1,patch_size, patch_size,dim3)
    m_net = WaveCnn(class_num,band_nums,patch_size)
    res = m_net(input)
    print('res',res.shape)

res torch.Size([1024, 16])


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelsum = WaveCnn(class_num,band_nums,patch_size).to(device)
# print(modelsum)
# summary(modelsum,(1,patch_size,patch_size,pca_components))
summary(modelsum,(1,patch_size,patch_size,dim3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 15, 15, 15, 1]             225
       BatchNorm3d-2        [-1, 15, 15, 15, 1]              30
              ReLU-3        [-1, 15, 15, 15, 1]               0
    BasicConvBlock-4        [-1, 15, 15, 15, 1]               0
  ReplicationPad3d-5        [-1, 1, 15, 15, 16]               0
            Conv3d-6        [-1, 1, 15, 15, 15]               2
            Conv3d-7        [-1, 1, 15, 15, 15]               2
           Wavelet-8  [[-1, 1, 15, 15, 15], [-1, 1, 15, 15, 15]]               0
  ReplicationPad3d-9        [-1, 1, 15, 16, 15]               0
           Conv3d-10        [-1, 1, 15, 15, 15]               2
           Conv3d-11        [-1, 1, 15, 15, 15]               2
          Wavelet-12  [[-1, 1, 15, 15, 15], [-1, 1, 15, 15, 15]]               0
 ReplicationPad3d-13        [-1, 1, 15, 16, 15]               0
     

In [17]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# net = WCNN(num_classes=16,wavelet='db1').to(device)
net = WaveCnn(class_num,band_nums,patch_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

patience = 30
early_stopping = EarlyStopping(patience,verbose=True)

start = time.time()
total_loss = 0
net.train() #注意启用训练模式
for epoch in range(200):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('[Epoch: %d]   [loss avg: %.6f]   [current loss: %.6f]' %(epoch + 1, total_loss/(epoch+1), loss.item()))

    early_stopping(loss,net)
    if early_stopping.early_stop:
        print("Early stopping")
        break

print('Finished Training')
end = time.time()
print('Training time: ', end-start)

[Epoch: 1]   [loss avg: 24.416433]   [current loss: 2.805534]
Validation loss decreased (inf --> 2.805534).  Saving model ...
[Epoch: 2]   [loss avg: 22.804754]   [current loss: 2.357686]
Validation loss decreased (2.805534 --> 2.357686).  Saving model ...
[Epoch: 3]   [loss avg: 21.880582]   [current loss: 2.479681]
EarlyStopping counter: 1 out of 30
[Epoch: 4]   [loss avg: 20.997437]   [current loss: 1.957170]
Validation loss decreased (2.357686 --> 1.957170).  Saving model ...
[Epoch: 5]   [loss avg: 20.013344]   [current loss: 1.594027]
Validation loss decreased (1.957170 --> 1.594027).  Saving model ...
[Epoch: 6]   [loss avg: 19.124608]   [current loss: 1.651574]
EarlyStopping counter: 1 out of 30
[Epoch: 7]   [loss avg: 18.326313]   [current loss: 1.518794]
Validation loss decreased (1.594027 --> 1.518794).  Saving model ...
[Epoch: 8]   [loss avg: 17.724023]   [current loss: 1.663420]
EarlyStopping counter: 1 out of 30
[Epoch: 9]   [loss avg: 17.154218]   [current loss: 1.57358

In [18]:
start = time.time()

count = 0
# 模型测试
net.eval()  #注意启用测试模式
for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = net(inputs)
    outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
    if count == 0:
        y_pred_test =  outputs
        count = 1
    else:
        y_pred_test = np.concatenate( (y_pred_test, outputs) )


        
end = time.time()
print('testing time: ', end-start)
        
# 生成分类报告
classification = classification_report(ytest, y_pred_test, digits=4)
print(classification)

testing time:  1.7966318130493164
              precision    recall  f1-score   support

         0.0     0.9965    0.9980    0.9972      1989
         1.0     0.9997    1.0000    0.9999      3689
         2.0     0.9765    0.9995    0.9879      1956
         3.0     0.9630    0.9986    0.9804      1380
         4.0     0.9992    0.9619    0.9802      2651
         5.0     0.9969    1.0000    0.9985      3920
         6.0     1.0000    0.9977    0.9989      3543
         7.0     0.8964    0.9792    0.9360     11158
         8.0     0.9998    0.9993    0.9996      6141
         9.0     0.9991    0.9864    0.9927      3245
        10.0     0.9940    0.9376    0.9649      1057
        11.0     0.9646    1.0000    0.9820      1908
        12.0     1.0000    0.7652    0.8670       907
        13.0     0.8224    0.9972    0.9014      1059
        14.0     0.9611    0.8275    0.8893      7196
        15.0     0.9972    0.9888    0.9930      1789

    accuracy                         0.9638   

In [19]:
from operator import truediv
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc
    
def reports (X_test,y_test,name):
    count = 0
    # 模型测试
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = net(inputs)
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        if count == 0:
            y_pred =  outputs
            count = 1
        else:
            y_pred = np.concatenate( (y_pred, outputs) )
    
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                          ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                         'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                          'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                          'Stone-Steel-Towers'] 
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    elif name == 'KSC':
        target_names = ['Water','Mud_flats','Salt_marsh','Catiail_marsh','Spartina_marsh','Graminoid_marsh',
                       'Hardwood_swamp','Oak-Broadleaf','Slash_Pine','CP-Oak','CP-hammock','Willow swamp',
                       'Scrub']
    
    classification = classification_report(y_test, y_pred, target_names=target_names)
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)

    return classification, confusion, oa*100, each_acc*100, aa*100, kappa*100

In [20]:
classification, confusion, oa, each_acc, aa, kappa = reports(test_loader, ytest, dataset)

print(oa)
print(aa)
print(kappa)

96.38165260879302
96.48074037246235
95.96450952138488
